# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232830403309                   -0.50    7.0
  2   -7.249548044294       -1.78       -1.39    1.0
  3   -7.251144386776       -2.80       -1.85    4.0
  4   -7.251082034025   +   -4.21       -1.95    3.0
  5   -7.251296902015       -3.67       -2.36    2.0
  6   -7.251333610066       -4.44       -2.75    2.0
  7   -7.251338705811       -5.29       -3.75    2.0
  8   -7.251338781703       -7.12       -3.92    4.0
  9   -7.251338795245       -7.87       -4.31    1.0
 10   -7.251338798484       -8.49       -4.87    2.0
 11   -7.251338798678       -9.71       -5.34    3.0
 12   -7.251338798703      -10.60       -5.88    3.0
 13   -7.251338798704      -11.85       -6.43    2.0
 14   -7.251338798705      -13.10       -6.73    3.0
 15   -7.251338798705      -13.43       -7.38    1.0
 16   -7.251338798705   +  -14.75       -7.79    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04929941482744436
[ Info: Arnoldi iteration step 2: normres = 0.5498278098205504
[ Info: Arnoldi iteration step 3: normres = 0.6733034082007715
[ Info: Arnoldi iteration step 4: normres = 0.5479701183576479
[ Info: Arnoldi iteration step 5: normres = 0.36160438120357963
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (8.16e-02, 6.35e-02, 2.36e-01, 2.53e-01, 2.18e-02)
[ Info: Arnoldi iteration step 6: normres = 0.41200460219642093
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.75e-02, 9.43e-02, 3.73e-01, 9.05e-02, 9.99e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09714327372534842
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.37e-03, 1.40e-02, 3.06e-02, 3.61e-02, 7.41e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08569311716819913
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.25e